In [ ]:
import onnxruntime as ort
import time
import numpy as np

BASE_DIR = "../exports/"

sess = ort.InferenceSession(
    BASE_DIR + "model_fp32.onnx",
    providers=["CUDAExecutionProvider"]
)

x = np.random.randn(1,3,224,224).astype(np.float32)

# warmup
for _ in range(20):
    sess.run(None, {"input": x})

# benchmark
t0 = time.time()
for _ in range(1000):
    sess.run(None, {"input": x})
print("ONNX FP32 latency:", (time.time()-t0)/1000)


/usr/local/lib/python3.12/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:123: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


NoSuchFile: [ONNXRuntimeError] : 3 : NO_SUCHFILE : Load model from ../exports/model_fp32.onnx failed:Load model ../exports/model_fp32.onnx failed. File doesn't exist

In [26]:
sess = ort.InferenceSession(
    BASE_DIR + "model_fp16.onnx",
    providers=["CUDAExecutionProvider"]
)

x = np.random.randn(1,3,224,224).astype(np.float16)

# benchmark
t0 = time.time()
for _ in range(1000):
    sess.run(None, {"input": x})
print("ONNX FP16 latency:", (time.time()-t0)/1000)

2025-12-14 16:31:47.521363605 [E:onnxruntime:Default, provider_bridge_ort.cc:2251 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1844 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcudnn.so.9: cannot open shared object file: No such file or directory

2025-12-14 16:31:47.521396178 [W:onnxruntime:Default, onnxruntime_pybind_state.cc:1013 CreateExecutionProviderFactoryInstance] Failed to create CUDAExecutionProvider. Require cuDNN 9.* and CUDA 12.*. Please install all dependencies as mentioned in the GPU requirements page (https://onnxruntime.ai/docs/execution-providers/CUDA-ExecutionProvider.html#requirements), make sure they're in the PATH, and that your GPU is supported.


ONNX FP16 latency: 0.002590458393096924


In [5]:
import numpy as np
from pathlib import Path

LABEL_DIR = Path("../data/CALIB")
PRED_DIR = Path("../data/TRT_OUT_FP16")

correct = 0
total = 0

for label_file in sorted(LABEL_DIR.glob("test_labels_*.npy")):
    idx = label_file.stem.split("_")[-1]

    y_true = np.load(label_file)
    y_pred = np.load(PRED_DIR / f"trt_preds_{idx}.npy")

    pred = y_pred.argmax(axis=1)
    pred = pred[:len(y_true)] 
    correct += (pred == y_true).sum()
    total += len(y_true)

print("TensorRT FP16 accuracy:", correct / total)


TensorRT FP16 accuracy: 0.9817073170731707


In [6]:
import numpy as np
from pathlib import Path

LABEL_DIR = Path("../data/CALIB")
PRED_DIR = Path("../data/TRT_OUT_INT8")

correct = 0
total = 0

for label_file in sorted(LABEL_DIR.glob("test_labels_*.npy")):
    idx = label_file.stem.split("_")[-1]

    y_true = np.load(label_file)
    y_pred = np.load(PRED_DIR / f"trt_preds_{idx}.npy")

    pred = y_pred.argmax(axis=1)
    pred = pred[:len(y_true)] 
    correct += (pred == y_true).sum()
    total += len(y_true)

print("TensorRT INT8 accuracy:", correct / total)


TensorRT INT8 accuracy: 0.6439024390243903
